In [10]:
! pwd
! pip uninstall sympytorch -y

/share_zeta/Proxy-Sim/PhysicsSimulationDeepLearning


Found existing installation: sympytorch 0.1.4
Uninstalling sympytorch-0.1.4:
  Successfully uninstalled sympytorch-0.1.4


In [1]:
import os
import sys


ROOT="/home/guillermo.carrilho/PhysicsSimulationDeepLearning"
#ROOT="/share_zeta/Proxy-Sim/PhysicsSimulationDeepLearning"

sys.path.append(os.path.join(ROOT,"Physical_models"))

Equation solve by this algorithm is:
$$\textbf{u}=-\frac{\textbf{k}}{\mu}(\nabla p - \rho g)$$

### The case of this implementation is

$$\phi \frac{\partial p}{\partial t} - \frac{1}{\mu} \nabla \cdot (\textbf{k} \nabla p) = q$$

# stable fluids approach

Equation solve by this algorithm is:
$$\textbf{u}=-\frac{\textbf{k}}{\mu}(\nabla p - \rho g)$$

### The case of this implementation is

$$\phi \frac{\partial p}{\partial t} - \frac{1}{\mu} \nabla \cdot (\textbf{k} \nabla p) = q$$

In [2]:
#from Differentiable_simulation import dK_w,K_w,K_o,grad_phi_dK
from phi.torch.flow import *
from phi.torch.flow import diffuse, advect, Solve, fluid, math,Field, unstack,stack,batch,field,vec

class darcy_flow_model(object):
  def __init__(self,field,dt):
    self.field=field
    self.dt=dt

  def nK(self,domain):
    return 0.001*math.exp(-math.sum(((domain.center-0.5)**2),"vector")/(2*0.5))

  def reaction_diffusion(self, u, dt):
    su = self.nK(u.geometry) * field.laplace(u)
    return u + dt * su
  def step(self,v):
    return self.reaction_diffusion(v,self.dt)
  
args={"k":0.1}
  

geo=UniformGrid(x=60, y=60,bounds=Box(x=1.0, y=1.0))
phi_w=Field( geo,values=vec(x=0.0,y=0.0),
      boundary= {
          'x-': vec(x=0.1,y=0.0),
          'x+': ZERO_GRADIENT,
          'y-': 0,
          'y+': 0
 })

#phy=physical_model(
#   phi_w,0.01
#)

## 2D

In [9]:
geo=UniformGrid(x=20, y=20,bounds=Box(x=1.0, y=1.0))
phi_w=Field( geo,values=tensor(0.0),
      boundary= {
          'x-': tensor(0.01),
          'x+': ZERO_GRADIENT,
          'y-': 0,
          'y+': 0
 })
phi_w
model=darcy_flow_model()

TypeError: darcy_flow_model.__init__() missing 2 required positional arguments: 'field' and 'dt'

In [3]:
class simple_cnn_model(torch.nn.Module):
  def __init__(self):
    super(simple_cnn_model,self).__init__()

    self.conv1=torch.nn.Conv2d(2, 32, (9,9), padding=4)
    self.conv2=torch.nn.Conv2d(32, 64, (9,9), padding=4)
    self.conv3=torch.nn.Conv2d(64, 2, (9,9), padding=4)
    self.act1=torch.nn.ReLU()
    self.act2=torch.nn.ReLU()
    self.act3=torch.nn.Tanh()
  def forward(self,x):
    x=self.act1(self.conv1(x))
    x=self.act2(self.conv2(x))
    x=self.act3(self.conv3(x))*1
    #x=torch.clamp(x, min=-0.5, max=0.5)
    return x[0]

In [4]:
model=simple_cnn_model().train()
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3)

In [12]:
v,p = fluid.make_incompressible(v0,solve=Solve('CG-adaptive',1e-2,1e-2))

NameError: name 'v0' is not defined

In [14]:
from Train_differentiable_physics import PINNS_based_SOL_trainer
import torch
T=PINNS_based_SOL_trainer(
    field=(v0,fluid.make_incompressible(v0,solve=Solve('CG-adaptive',1e-2,1e-2))[-1]),
    model=model,
    optimizer=optimizer,
    simulation_steps=2,
    time_step=0.1,
    loss=torch.mean
    )

In [16]:
from random import randint
randint()

TypeError: Random.randint() missing 2 required positional arguments: 'a' and 'b'

In [15]:
T.train(5)

[array(0.5626921, dtype=float32),
 array(-0.45075464, dtype=float32),
 array(-0.6038111, dtype=float32),
 array(-0.64447063, dtype=float32),
 array(-0.6510437, dtype=float32)]

## 3D

/usr/local/lib/python3.10/dist-packages/phiml/math/_optimize.py:640: UserWarning: Possible rank deficiency detected. Matrix might be singular which can lead to convergence problems. Please specify using Solve(rank_deficiency=...).
  warnings.warn("Possible rank deficiency detected. Matrix might be singular which can lead to convergence problems. Please specify using Solve(rank_deficiency=...).")


Diverged: Solve diverged within 819 iterations using Φ-ML CG (PyTorch*)

In [13]:
boundary = {'x-': vec(x=1.0,y=0.0,z=0.0),
            'x+': ZERO_GRADIENT,
            'y-':0,
            'y+':0,
            'z-':0,
            'z+':0
            }
boundary
geo=UniformGrid(x=128, y=128,z=128,bounds=Box(x=50, y=50,z=50))
v0=Field(geo,values=vec(x=tensor(0.0),y=tensor(0.0),z=tensor(0.0)),boundary=boundary)

phys=darcy_flow_model(v0,dt=0.1,
                    #advection_solver=(lambda v: Solve('CG-adaptive',1e-2,1e-2,x0=v)),projection_solver=Solve('CG-adaptive',1e-2,1e-2)
                    )

In [14]:
phys.step(v0)

Field[(xˢ=128, yˢ=128, zˢ=128, vectorᶜ=x,y,z)]

In [15]:
v0.geometry.

(xˢ=128, yˢ=128, zˢ=128), bounds=Box(x=(0, 50), y=(0, 50), z=(0, 50))

# Differentiable physics with PINNs

## Linear Darcy model

with
$$K(x)=\exp((p-p_0)^2)$$

### 2D

#### Solver

In [5]:
from phi.torch.flow import *
class darcy_flow_model(object):
  def __init__(self,field,dt):
    self.field=field
    self.dt=dt

  def nK(self,domain):
    return 0.001*math.exp(-math.sum(((domain.center-0.5)**2),"vector")/(2*0.5))

  def reaction_diffusion(self, u, dt):
    su = self.nK(u.geometry) * field.laplace(u)
    return u + dt * su
  def step(self,v):
    return self.reaction_diffusion(v,self.dt)
args={"k":0.1}


In [6]:
geo=UniformGrid(x=20, y=20,bounds=Box(x=1.0, y=1.0))
phi_w=Field( geo,values=tensor(0.0),
      boundary= {
          'x-': tensor(0.01),
          'x+': ZERO_GRADIENT,
          'y-': 0,
          'y+': 0
 })
phi_w



model=darcy_flow_model(phi_w,0.1)

In [7]:
phi_w.geometry.center.native("x,y").shape

torch.Size([20, 20, 2])

In [8]:
PHI=[]
for i in range(100):
  phi_w=model.reaction_diffusion(phi_w,0.01)
  PHI.append(phi_w)

In [ ]:
for i in PHI[::10]:
  plot(i)

#### Solver in the loop

In [9]:
from Train_differentiable_physics_with_PINNs import PINNS_based_SOL_trainer

In [10]:
import torch


def derivatives(u,x,n):
    if n==0:
        return u
    else:
        du=torch.autograd.grad(u,x,
        grad_outputs=torch.ones_like(u).to(u.device),
        create_graph=True,
        retain_graph=True,
        allow_unused=True
        )[0]
    return derivatives(du,x,n-1)


def vector_jacobian(u,x):
    unit_vectors=torch.eye(u.shape[-1])
    jacobian_rows = [torch.autograd.grad(u, x, vec_.unsqueeze(0).unsqueeze(0).tile(u.shape[0],u.shape[1],1).to(u.device),
            create_graph=True,
            retain_graph=True,
            allow_unused=True)[0]
                     for vec_ in unit_vectors]
    return torch.stack(jacobian_rows,axis=2)


def vector_grad(u,x):
    unit_vectors=torch.eye(u.shape[-1])
    jacobian_rows = [torch.autograd.grad(u, x, vec_.unsqueeze(0).unsqueeze(0).tile(u.shape[0],u.shape[1],1).to(u.device),
            create_graph=True,
            retain_graph=True,
            allow_unused=True)[0]
                     for vec_ in unit_vectors]
    return torch.diagonal(torch.stack(jacobian_rows,axis=2),dim1=-2,dim2=-1)

def x_grad(u,x,i,n):
    """
    gradient of degree n wrt x for component i of u
    input:
    u and x are tensors with vectors object at dimension -1
    [b, n_vectors, vector_dimension]

    output:
    [b, n_vectors, input_vector_dimension]
    """
    I=torch.eye(u.shape[-1])

    u=torch.autograd.grad(u ,x,
            I[i].unsqueeze(0).unsqueeze(0).tile(u.shape[0],u.shape[1],1).to(u.device),
            create_graph=True,
            retain_graph=True,
            allow_unused=True)[0]
    if n > 1:
        for i in range(n-1):
            u=vector_grad(u,x)
    return u

In [11]:
import torch
from einops import rearrange
import matplotlib.pyplot as plt

class simple_cnn_model(torch.nn.Module):
  def __init__(self):
    super(simple_cnn_model,self).__init__()

    self.conv1=torch.nn.Conv2d(4, 32, (9,9), padding=4)
    self.conv2=torch.nn.Conv2d(32, 64, (9,9), padding=4)
    self.conv3=torch.nn.Conv2d(64, 1, (9,9), padding=4)
    self.act1=torch.nn.ReLU()
    self.act2=torch.nn.ReLU()
    self.act3=torch.nn.Tanh()
  def forward(self,x):
    x=self.act1(self.conv1(x))
    x=self.act2(self.conv2(x))
    x=self.act3(self.conv3(x))*0.01
    #x=torch.clamp(x, min=-0.5, max=0.5)
    return x
  

#class composed_loss(object):

#from DL_models.PINNS.utils import x_grad

def composed_loss(x,p):
  """
  u: [b x y p]
  x: [b x y (x y t)]
  """
  #p=rearrange(p,"b p x y-> b (x y) p")
  #x=rearrange(x,"b u x y-> b (x y) u")
  dpdt=x_grad(p,x,0,1)[...,2]
  lap_u=torch.sum(x_grad(p,x,0,2)[...,:2],axis=-1)
  k=0.001*torch.exp(
    -torch.sum(((x[...,:2]-torch.mean(x[...,:2]))**2),"vector")/(2*0.5))

  return torch.mean(dpdt-k*lap_u)

/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [12]:
st_model=simple_cnn_model().train()
optimizer=torch.optim.Adam(st_model.parameters(),lr=1e-4)

In [13]:
geo

(xˢ=20, yˢ=20), bounds=Box(x=(0, 1.0), y=(0, 1.0))

In [14]:
from random import randint
from Physical_models.Differentiable_simulation import Space2Tensor,Tensor2Space


class PINNS_based_SOL_trainer(PINNS_based_SOL_trainer):
      
    def generate_postion_time_code(self,field,t):  # Re implement if more dimensions are needed
      X=field.geometry.center.native("x,y")
      T=torch.ones(X.shape[:2]+(1,))*t
      XT=rearrange(torch.concat((X,T),axis=-1),"x y c -> c x y").unsqueeze(0)
      return XT

    def correct(self,states_pred):
      # TODO: implement a method to be re implemented for other architecures
      XTUp=rearrange(self.st_model(rearrange(states_pred," b (x y) u -> b u x y",x=self.os_[0],y=self.os_[1])),"b u x y  -> b (x y) u ")
      # GRAD HERE
      #x_grad(u,x.reshape(2,4,-1).transpose(1,-1),0,0)
      return states_pred,XTUp


    def forward_prediction_correction(self):

      # FIELD FORMAT
      self.os_=self.v.geometry.center.native("x,y").shape
      # SQUARED TENSOR FORMAT
      XT=self.generate_postion_time_code(self.v,self.t)
      XTU=torch.concat((XT,Space2Tensor(self.v,self.geometry)),axis=1)

      # TABULAR TENSOR FORMAT
      states_pred=[rearrange(XTU,"b u x y-> b (x y) u").requires_grad_(True)]  # START OF FLUX
      XTUp_1,XTUp=self.correct(states_pred[-1])
      #print("first grad")
      #print(x_grad(XTUp,states_pred,0,1))
      
      # OBS: correction is made in field forrmat, for no aparent reason (TODO: try to train in field format)
      states_in=[XTUp_1]
      states_corr=[XTUp]
      states_pred=[XTUp_1[:,:,-1:]+XTUp]
      # GRAD HERE
      #print("correction grad")
      #print(x_grad(XTUp_1[:,:,-1:]+XTUp,states_in[0],0,1))

      # For steps in correction run (4 in example) (incidencia nos iniciais)
      for i in range(self.n_steps):
        
        # Step last in states_pred
        U_space=Tensor2Space(rearrange(states_pred[-1]," b (x y) u -> b u x y",x=self.os_[0],y=self.os_[1])[0],self.geometry)
        states_pred.append(self.ph_model.step(
          U_space
          ))
        #U_tensor=Space2Tensor(U_space,self.geometry)
        self.t=self.t+self.dt
        XT=self.generate_postion_time_code(U_space,self.t)
        XTU=torch.concat((XT,Space2Tensor(U_space,self.geometry)),axis=1)
        # GRAD HERE
        
        states_pred[-1]=rearrange(XTU,"b u x y-> b (x y) u")
        # Correct with model of last states_pred
        XTUp_1,XTUp=self.correct(states_pred[-1])

        states_in.append(XTUp_1)
        states_corr.append(XTUp)
        states_pred[-1]=XTUp_1[:,:,-1:]+XTUp

      return states_pred,states_corr,states_in
    
    def detach_field(self,field):
      tensor=Space2Tensor(field,self.geometry).detach()
      return Tensor2Space(tensor[0],self.geometry)
    
    def train(self,epochs):
      losses=[]
      for i in range(epochs):
        print(f"epoch {i}")
        co_batch=[]
        in_batch=[]
        for b in range(2):
          print(f"batch {b}")

          random_idx=randint(0,len(self.init_states_gt)-1)
          self.v=self.detach_field(self.init_states_gt[random_idx])
          self.t=self.T[random_idx]
          states_pred,states_corr,states_in=self.forward_prediction_correction()
          #print("before last grad, in batch")
          #print(print(x_grad(states_pred[0],states_in[0],0,1)))
          #print(len(states_in))
          #for p,i in zip(states_pred,states_in):
          #  print("before last grad, in batch")
          #  print(print(x_grad(p,i,0,1)))
          co_batch=co_batch+states_pred
          in_batch=in_batch+states_in

        #print("just before concat")
        #print(x_grad(states_pred[0],states_in[0],0,1))
        for p,i in zip(states_pred,states_in):
          print(p.shape)
          print("before last grad, in batch")
          print(x_grad(p,i,0,1))
        states_in=torch.cat(in_batch,axis=0) # [B X Y U]
        print(states_in.shape)
        states_pred=torch.cat(co_batch,axis=0) # [B X Y U]
        print(states_pred.shape)
        print("last grad after concat")
        print(print(x_grad(states_pred,states_in,0,1)))
        print("LOSS")
        loss=self.loss(states_in,states_pred)


        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        losses.append(loss.cpu().detach().numpy())
      return losses

In [15]:
phi_w.geometry.center.native("x,y").shape

torch.Size([20, 20, 2])

In [16]:
T=PINNS_based_SOL_trainer(
    field=phi_w,
    physical_model=darcy_flow_model,
    statistical_model=st_model,
    optimizer=optimizer,
    simulation_steps=3,
    time_step=0.1,
    loss=composed_loss
)

In [17]:
T.train(2)

epoch 0
batch 0
first grad
tensor([[[ 3.4492e-04, -1.0512e-04, -8.8167e-06, -1.8425e-04],
         [ 2.8393e-04, -2.1647e-04, -2.0530e-04, -2.4987e-04],
         [ 5.0655e-04, -3.3910e-04, -3.7305e-05, -4.6391e-04],
         ...,
         [-1.9095e-04,  1.0800e-04, -5.0366e-05,  1.4076e-04],
         [-1.7541e-04,  3.1264e-06, -2.7305e-04,  3.7667e-04],
         [-1.7862e-04,  8.6527e-05, -3.7979e-04,  8.8205e-05]]],
       grad_fn=<ReshapeAliasBackward0>)
correction grad
tensor([[[ 3.4492e-04, -1.0512e-04, -8.8167e-06,  9.9982e-01],
         [ 2.8393e-04, -2.1647e-04, -2.0530e-04,  9.9975e-01],
         [ 5.0655e-04, -3.3910e-04, -3.7305e-05,  9.9954e-01],
         ...,
         [-1.9095e-04,  1.0800e-04, -5.0366e-05,  1.0001e+00],
         [-1.7541e-04,  3.1264e-06, -2.7305e-04,  1.0004e+00],
         [-1.7862e-04,  8.6527e-05, -3.7979e-04,  1.0001e+00]]],
       grad_fn=<AddBackward0>)
batch 1
first grad
tensor([[[ 2.9140e-04, -1.2502e-04, -1.3198e-04, -1.6752e-04],
         [ 2.918

TypeError: 'NoneType' object is not subscriptable

In [15]:
#x=
#x=rearrange(torch.randint(0,4,(2,4,50,50),requires_grad=True,dtype=torch.float),"b u x y-> b (x y) u")
st_model=simple_cnn_model()
x=torch.randint(0,4,(2,2500,4),requires_grad=True,dtype=torch.float)

x=math.exp(x)

u=st_model(rearrange(x,"b (x y) u-> b u x y",x=50,y=50)).unsqueeze(0)
#u=st_model(x.reshape(2,4,50,50))
print(x.shape)
print(u.shape)

u=rearrange(u,"b p x y-> b (x y) p")
#u=u.reshape(2,1,-1).transpose(1,-1)

#x=x.transpose(1,-1)

print(x.shape)
print(u.shape)
x_grad(u,x,0,0)

x_grad(u,x.reshape(2,4,-1).transpose(1,-1),0,0)

torch.Size([2, 2500, 4])
torch.Size([1, 1, 50, 50])
torch.Size([2, 2500, 4])
torch.Size([1, 2500, 1])


In [68]:
class darcy_flow_model(object):
  def __init__(self,field,dt):
    self.field=field
    self.dt=dt

  def nK(self,domain):
    return 0.001*math.exp(-math.sum(((domain.center-0.5)**2),"vector")/(2*0.5))

  def reaction_diffusion(self, u, dt):
    su = self.nK(u.geometry) * field.laplace(u)
    return u + dt * su
  def step(self,v):
    return self.reaction_diffusion(v,self.dt)
model=darcy_flow_model(phi_w,0.1)

import torch
from phi.torch.flow import *
from DL_models.PINNS.utils import x_grad
from Physical_models.Differentiable_simulation import Space2Tensor,Tensor2Space

def generate_postion_time_code(field,t):  # Re implement if more dimensions are needed
    X=field.geometry.center.native("x,y")
    T=torch.ones(X.shape[:2]+(1,))*t
    XT=rearrange(torch.concat((X,T),axis=-1),"x y c -> c x y").unsqueeze(0)
    return XT


class simple_cnn_model(torch.nn.Module):
  def __init__(self):
    super(simple_cnn_model,self).__init__()

    self.conv1=torch.nn.Conv2d(7, 32, (9,9), padding=4)
    self.conv2=torch.nn.Conv2d(32, 64, (9,9), padding=4)
    self.conv3=torch.nn.Conv2d(64, 1, (9,9), padding=4)
    self.act1=torch.nn.ReLU()
    self.act2=torch.nn.ReLU()
    self.act3=torch.nn.Tanh()
  def forward(self,x):
    x=self.act1(self.conv1(x))
    x=self.act2(self.conv2(x))
    x=self.act3(self.conv3(x))*0.01
    #x=torch.clamp(x, min=-0.5, max=0.5)
    return x[0]

In [72]:
st_model=simple_cnn_model()
x=torch.randint(0,4,(1,2500,4),requires_grad=True,dtype=torch.float)

xs=Tensor2Space(rearrange(x,"b (x y) u-> b u x y",x=50,y=50)[0],UniformGrid(x=50, y=50,bounds=Box(x=1.0, y=1.0)),'c x y-> x y c')

xs=model.step(
    xs
    )
xt=Space2Tensor(xs,UniformGrid(x=50, y=50,bounds=Box(x=1.0, y=1.0)),'x,y,vector')
u=st_model(torch.concat(
        (xt,generate_postion_time_code(Tensor2Space(rearrange(x,"b (x y) u-> b u x y",x=50,y=50)[0],UniformGrid(x=50, y=50,bounds=Box(x=1.0, y=1.0)),'c x y-> x y c'),0.1)),
        axis=1
        )).unsqueeze(0)
u=u+xt
print(xt.shape)
xs=Tensor2Space(u[0],UniformGrid(x=50, y=50,bounds=Box(x=1.0, y=1.0)),'c x y-> x y c')
xs=model.step(xs)
xt=Space2Tensor(xs,UniformGrid(x=50, y=50,bounds=Box(x=1.0, y=1.0)),'x,y,vector')
#x=math.exp(x)
u=st_model(torch.concat(
        (xt,generate_postion_time_code(Tensor2Space(rearrange(x,"b (x y) u-> b u x y",x=50,y=50)[0],UniformGrid(x=50, y=50,bounds=Box(x=1.0, y=1.0)),'c x y-> x y c'),0.1)),
        axis=1
        )).unsqueeze(0)
u=u+xt
##u=st_model(x.reshape(2,4,50,50))
#print(x.shape)
#print(u.shape)
#
u=rearrange(u,"b p x y-> b (x y) p")
##u=u.reshape(2,1,-1).transpose(1,-1)
#
##x=x.transpose(1,-1)
#
#print(x.shape)
#print(u.shape)
x_grad(u,x,0,0)
#
#x_grad(u,x.reshape(2,4,-1).transpose(1,-1),0,0)

torch.Size([1, 4, 50, 50])


tensor([[[ 5.3229e-01,  1.1484e-04, -2.5526e-04,  1.4773e-04],
         [ 7.1610e-01,  1.2463e-04, -3.0369e-04,  1.7276e-04],
         [ 7.3567e-01,  1.2197e-04, -2.8085e-04,  2.2035e-04],
         ...,
         [ 7.3543e-01, -1.5188e-04, -2.6878e-04,  3.1183e-05],
         [ 7.1576e-01, -1.1411e-04, -2.4443e-04, -5.9107e-05],
         [ 5.3207e-01,  6.9688e-06, -1.1721e-04, -9.2476e-05]]],
       grad_fn=<ReshapeAliasBackward0>)

In [70]:
torch.concat(
        (xt,generate_postion_time_code(Tensor2Space(rearrange(x,"b (x y) u-> b u x y",x=50,y=50)[0],UniformGrid(x=50, y=50,bounds=Box(x=1.0, y=1.0)),'c x y-> x y c'),0.1)),
        axis=1
        ).shape

torch.Size([1, 7, 50, 50])

In [61]:
generate_postion_time_code(Tensor2Space(rearrange(x,"b (x y) u-> b u x y",x=50,y=50)[0],UniformGrid(x=50, y=50,bounds=Box(x=1.0, y=1.0)),'c x y-> x y c'),0.1).shape

torch.Size([1, 3, 50, 50])

In [33]:
print(x.shape)
print(xs)
print(xt.shape)
u

torch.Size([1, 2500, 4])
Field[(xˢ=50, yˢ=50, vectorᶜ=4)]
torch.Size([1, 4, 50, 50])


tensor([[[[ 2.4456e-04, -3.6087e-05,  2.6534e-05,  ..., -4.7920e-04,
           -1.6923e-04,  1.2456e-04],
          [-7.8309e-04, -1.0338e-03, -6.9462e-04,  ..., -1.1733e-03,
           -4.3531e-04,  3.0210e-04],
          [-5.7426e-04, -9.1421e-04,  9.6639e-04,  ..., -1.2151e-03,
           -1.4550e-03, -3.1069e-04],
          ...,
          [-9.1590e-04, -3.0385e-04, -6.5880e-04,  ..., -9.1798e-04,
            1.0899e-03,  1.4357e-03],
          [-2.9862e-04,  6.3538e-04,  6.8159e-04,  ...,  9.2079e-04,
            5.1439e-04,  7.3152e-04],
          [ 1.1529e-04,  4.3320e-04,  4.4886e-04,  ...,  5.6771e-04,
            7.5802e-04,  5.6851e-04]]]], grad_fn=<UnsqueezeBackward0>)

In [20]:
#geo=UniformGrid(x=50, y=50,bounds=Box(x=1.0, y=1.0))
#phi_w=Field(geo, values=tensor(rearrange(x,"b (x y) u-> b u x y",x=50,y=50),),
#      boundary= {
#          'x-': tensor(0.01),
#          'x+': ZERO_GRADIENT,
#          'y-': 0,
#          'y+': 0
# })

Tensor2Space

NameError: name 'Tensor2Space' is not defined

In [18]:

Field?

Init signature:
Field(
    geometry: Union[phi.geom._geom.Geometry, phiml.math._tensors.Tensor],
    values: Union[phiml.math._tensors.Tensor, numbers.Number, bool, Callable, phi.field._field.FieldInitializer, phi.geom._geom.Geometry, ForwardRef('Field')],
    boundary: Union[numbers.Number, phiml.math.extrapolation.Extrapolation, ForwardRef('Field'), dict] = 0.0,
    variable_attrs=('values',),
    value_attrs=('values',),
    **sampling_kwargs,
)
Docstring:     
A `Field` represents a discretized physical quantity (like temperature field or velocity field).
The sample points and their relation are encoded in the `geometry` property and the corresponding values are stored as one `Tensor` in `values`.
The boundary conditions and values outside the geometry are determined by `boundary`.

Examples:
    Create a periodic 2D grid, initialized via noise fluctuations.
    >>> Field(UniformGrid(x=32, y=32), values=Noise(), boundary=PERIODIC)

    Create a field on an unstructured mesh loaded 